In [1]:
import folium
import gpxpy
from streamlit_folium import folium_static

print("✅ 모든 라이브러리 설치 완료!")
print(f"Folium 버전: {folium.__version__}")
print(f"GPXpy 버전: {gpxpy.__version__}")

2025-11-03 16:13:23.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ 모든 라이브러리 설치 완료!
Folium 버전: 0.20.0
GPXpy 버전: 1.6.2


In [2]:
# GPX 파일 열기
with open('2025_JTBC.gpx', 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

print(f"트랙 개수: {len(gpx.tracks)}")
print(f"트랙 이름: {gpx.tracks[0].name}")

트랙 개수: 1
트랙 이름: 2025 jtbc marathon


In [4]:
import gpxpy

def extract_coordinates_detailed(gpx_file_path):
    """GPX 파일에서 좌표를 추출하는 상세 설명"""
    
    # 1단계: 파일 열기
    with open('2025_JTBC.gpx', 'r', encoding='utf-8') as gpx_file:
        # 2단계: GPX 파싱 (XML을 Python 객체로 변환)
        gpx = gpxpy.parse(gpx_file)
    
    # 좌표를 저장할 리스트
    coordinates = []
    
    # 3단계: 트랙 순회
    # gpx.tracks = GPX 파일 안의 모든 트랙 리스트
    for track in gpx.tracks:
        print(f"트랙 이름: {track.name}")
        
        # 4단계: 세그먼트 순회
        # track.segments = 트랙 안의 모든 세그먼트 리스트
        for segment in track.segments:
            print(f"  세그먼트 포인트 개수: {len(segment.points)}")
            
            # 5단계: 포인트 순회
            # segment.points = 세그먼트 안의 모든 GPS 포인트
            for point in segment.points:
                # 6단계: [위도, 경도] 형식으로 저장
                coordinates.append([
                    point.latitude,   # 위도 (예: 37.5665)
                    point.longitude   # 경도 (예: 126.9780)
                ])
    
    return coordinates

# 사용 예시
coords = extract_coordinates_detailed('marathon.gpx')
print(f"총 포인트 개수: {len(coords)}")
print(f"첫 번째 좌표: {coords[0]}")
print(f"마지막 좌표: {coords[-1]}")

트랙 이름: 2025 jtbc marathon
  세그먼트 포인트 개수: 4020
총 포인트 개수: 4020
첫 번째 좌표: [37.566226702183485, 126.89761410467327]
마지막 좌표: [37.51368306577206, 127.12367394939065]


In [5]:
def extract_full_data(gpx_file_path):
    """좌표, 고도, 시간을 모두 추출"""
    
    with open('2025_JTBC.gpx', 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    
    data = []
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append({
                    'lat': point.latitude,
                    'lon': point.longitude,
                    'elevation': point.elevation if point.elevation else 0,
                    'time': point.time
                })
    
    return data

# 사용
data = extract_full_data('marathon.gpx')
print(f"첫 번째 포인트:")
print(f"  위도: {data[0]['lat']}")
print(f"  경도: {data[0]['lon']}")
print(f"  고도: {data[0]['elevation']}m")
print(f"  시간: {data[0]['time']}")

첫 번째 포인트:
  위도: 37.566226702183485
  경도: 126.89761410467327
  고도: 14.0m
  시간: 2025-11-03 06:58:50+00:00


In [6]:
import folium

# 지도 생성 (서울 시청 중심)
m = folium.Map(
    location=[37.5665, 126.9780],  # [위도, 경도]
    zoom_start=13                   # 줌 레벨 (1~18)
)

# HTML 파일로 저장
m.save('simple_map.html')
print("지도가 생성되었습니다! simple_map.html 파일을 열어보세요")

지도가 생성되었습니다! simple_map.html 파일을 열어보세요


In [8]:
# 다양한 타일 스타일
styles = [
    'OpenStreetMap',      # 기본 (무료)
    'CartoDB positron',   # 밝은 테마 (추천!)
    'CartoDB dark_matter',# 어두운 테마
    'Stamen Terrain',     # 지형 강조
    'Stamen Watercolor'   # 수채화 느낌
]

# 밝은 테마 지도
m = folium.Map(
    location=[37.5665, 126.9780],
    zoom_start=13,
    tiles='CartoDB positron'  # 깔끔한 밝은 테마
)

# 기본 마커
folium.Marker(
    location=[37.5665, 126.9780],
    popup='서울 시청',           # 클릭하면 나타나는 텍스트
    tooltip='여기를 클릭하세요'   # 마우스 올리면 나타나는 텍스트
).add_to(m)

# 색상 있는 마커
folium.Marker(
    location=[37.5700, 126.9850],
    popup='광화문',
    icon=folium.Icon(
        color='red',      # 색상: blue, red, green, purple, orange, etc.
        icon='star'       # 아이콘: info-sign, star, heart, etc.
    )
).add_to(m)

# 원형 마커
folium.CircleMarker(
    location=[37.5750, 126.9900],
    radius=10,                    # 반지름
    popup='남산',
    color='blue',                 # 테두리 색
    fill=True,
    fillColor='lightblue',        # 채우기 색
    fillOpacity=0.6               # 투명도
).add_to(m)

m.save('markers_map.html')

In [9]:
import folium
import gpxpy

def create_marathon_map(gpx_file_path):
    """
    GPX 파일로부터 마라톤 지도 생성
    
    Args:
        gpx_file_path: GPX 파일 경로
    
    Returns:
        folium.Map: 생성된 지도 객체
    """
    
    # 1단계: GPX 파일에서 좌표 추출
    with open('2025_JTBC.gpx', 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    
    coordinates = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coordinates.append([point.latitude, point.longitude])
    
    print(f"✅ 총 {len(coordinates)}개의 GPS 포인트를 읽었습니다.")
    
    # 2단계: 지도 중심점 계산
    center_lat = sum([c[0] for c in coordinates]) / len(coordinates)
    center_lon = sum([c[1] for c in coordinates]) / len(coordinates)
    
    print(f"📍 지도 중심: ({center_lat:.4f}, {center_lon:.4f})")
    
    # 3단계: 지도 생성
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='CartoDB positron'  # 깔끔한 스타일
    )
    
    # 4단계: 코스 라인 그리기
    folium.PolyLine(
        locations=coordinates,
        color='#FF4444',          # 빨간색
        weight=6,                 # 선 두께
        opacity=0.8,              # 약간 투명
        popup='마라톤 코스 42.195km',
        tooltip='클릭하면 정보 표시'
    ).add_to(m)
    
    # 5단계: 출발점 마커
    folium.Marker(
        location=coordinates[0],
        popup='🏁 출발점<br>서울 시청',
        tooltip='출발!',
        icon=folium.Icon(
            color='green',
            icon='play',
            prefix='fa'  # Font Awesome 아이콘
        )
    ).add_to(m)
    
    # 6단계: 도착점 마커
    folium.Marker(
        location=coordinates[-1],
        popup='🎯 도착점<br>완주!',
        tooltip='피니시!',
        icon=folium.Icon(
            color='red',
            icon='stop',
            prefix='fa'
        )
    ).add_to(m)
    
    # 7단계: 10km마다 거리 마커 추가
    total_points = len(coordinates)
    for km in [10, 20, 21.0975, 30, 40]:
        # 포인트 인덱스 계산 (비율로)
        idx = int((km / 42.195) * total_points)
        if idx < total_points:
            folium.CircleMarker(
                location=coordinates[idx],
                radius=8,
                popup=f'{km}km 지점',
                color='blue',
                fill=True,
                fillColor='lightblue',
                fillOpacity=0.7
            ).add_to(m)
    
    print("✅ 지도 생성 완료!")
    return m

# 사용 예시
map_obj = create_marathon_map('seoul_marathon.gpx')
map_obj.save('marathon_map.html')
print("📁 marathon_map.html 파일을 열어보세요!")

✅ 총 4020개의 GPS 포인트를 읽었습니다.
📍 지도 중심: (37.5350, 127.0285)
✅ 지도 생성 완료!
📁 marathon_map.html 파일을 열어보세요!
